# reschedule

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [2]:
crop = pd.read_csv("/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv")
crop.head()


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [3]:
crop.isnull().sum()
crop.duplicated().sum()

0

In [4]:
print(crop.dtypes)

N                int64
P                int64
K                int64
temperature    float64
humidity       float64
ph             float64
rainfall       float64
label           object
dtype: object


In [5]:
unique_values = crop['label'].unique()
print(unique_values)


['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


In [6]:
# Map crop labels to numerical values
crop_dict = {
    'rice': 1,
    'maize': 2,
    'jute': 3,
    'cotton': 4,
    'coconut': 5,
    'papaya': 6,
    'orange': 7,
    'apple': 8,
    'muskmelon': 9,
    'watermelon': 10,
    'grapes': 11,
    'mango': 12,
    'banana': 13,
    'pomegranate': 14,
    'lentil': 15,
    'blackgram': 16,
    'mungbean': 17,
    'mothbeans': 18,
    'pigeonpeas': 19,
    'kidneybeans': 20,
    'chickpea': 21,
    'coffee': 22
}
crop['crop_num'] = crop['label'].map(crop_dict)
crop['crop_num'].value_counts()

crop_num
1     100
2     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
11    100
12    100
13    100
14    100
15    100
16    100
17    100
18    100
19    100
20    100
21    100
22    100
Name: count, dtype: int64

In [7]:
X = crop.drop(['crop_num', 'label'], axis=1)
Y = crop['crop_num']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [9]:
ms = MinMaxScaler()
sc = StandardScaler()


In [10]:
# Fit and transform the training data
X_train = ms.fit_transform(X_train)
X_test = ms.transform(X_test)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [11]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'Support Vector Machine': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Bagging': BaggingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Extra Trees': ExtraTreeClassifier(),
}

In [12]:
# Train and evaluate models
for name, md in models.items():
    md.fit(X_train, y_train)
    ypred = md.predict(X_test)
    print(f"{name} with accuracy: {accuracy_score(y_test, ypred)}")


Logistic Regression with accuracy: 0.9636363636363636
Naive Bayes with accuracy: 0.9954545454545455
Support Vector Machine with accuracy: 0.9681818181818181
K-Nearest Neighbors with accuracy: 0.9590909090909091
Decision Tree with accuracy: 0.9886363636363636
Random Forest with accuracy: 0.9931818181818182
Bagging with accuracy: 0.9886363636363636
AdaBoost with accuracy: 0.1409090909090909
Gradient Boosting with accuracy: 0.9818181818181818
Extra Trees with accuracy: 0.9227272727272727


In [13]:
# Train the final model (Random Forest Classifier)
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
ypred = rfc.predict(X_test)
print(f"Random Forest Classifier accuracy: {accuracy_score(y_test, ypred)}")


Random Forest Classifier accuracy: 0.9931818181818182


In [15]:
def recommendation(N, P, k, temperature, humidity, ph, rainfall):
    features = np.array([[N, P, k, temperature, humidity, ph, rainfall]])
    transformed_features = ms.transform(features)
    transformed_features = sc.transform(transformed_features)
    prediction = rfc.predict(transformed_features)
    return prediction[0]

In [16]:
N = float(input("Nitrogen: "))
P = float(input("Phosphorus: "))
k = float(input("Potassium: "))
humidity = float(input("Humidity: "))
temperature = float(input("Temperature: "))
ph = float(input("pH: "))
rainfall = float(input("Rainfall: "))
predict = recommendation(N, P, k, temperature, humidity, ph, rainfall)

# Map prediction to crop name
crop_dict_rev = {v: k for k, v in crop_dict.items()}
if predict in crop_dict_rev:
    crop_name = crop_dict_rev[predict]
    print("{} is the best crop to be cultivated".format(crop_name))
else:
    print("Sorry, we are not able to recommend a proper crop for this environment")

# Save models and scalers
import pickle
pickle.dump(rfc, open('model.pkl', 'wb'))
pickle.dump(ms, open('minmaxscaler.pkl', 'wb'))
pickle.dump(sc, open('standscaler.pkl', 'wb'))

Nitrogen:  90
Phosphorus:  42
Potassium:  43
Humidity:  20.8
Temperature:  82
pH:  6.5
Rainfall:  202


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


TypeError: unhashable type: 'numpy.ndarray'